# Packages loading

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from time import sleep 
import time
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import json
import random

# Scraping first try

In [2]:
with open('articles.json') as f:
    FINAL  = json.load(f)

In [3]:
proxy_list = [
    '79.78.184.12:56834',
    '196.40.188.137:41423',
    '134.196.244.120:46425',
    '186.192.49.60:8080',
    '103.229.177.51:48072',
    '157.230.33.168:3128',
    '202.79.48.38:56079',
    '128.199.226.214:8080',
    '142.93.104.92:8118',
    '49.156.42.188:8080',
    '66.251.139.85:8080',
    '41.90.115.18:61733',
    '124.120.129.38:80',
    '80.87.217.6:8080',
    '122.143.87.23:8080',
    '23.239.18.176:3128',
    '67.60.137.219:35979',
    '35.188.253.121:3128',
    '185.20.163.132:8080',
    '62.176.27.228:3128',
    '97.75.124.126:8080',
    '123.231.226.114:47562',
    '47.90.200.53:3128',
    '202.158.49.140:56687',
    '159.65.185.6:8888',
    '50.84.27.59:8080',   
]

In [4]:
def Start(startPage, proxy):
    # Открываем браузер, вводим логин и пароль
    if proxy != None:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--proxy-server=%s' % proxy)
        driver = webdriver.Chrome(executable_path='/home/skvid/Desktop/сети/project_2/chromedriver', options=chrome_options)
    else:
        driver = webdriver.Chrome(executable_path='/home/skvid/Desktop/сети/project_2/chromedriver')
        
    driver.get("http://proxylibrary.hse.ru:2048/login?url=http://isiknowledge.com/wos")
    inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[1]/td[2]/input")
    inputElement.send_keys('marishamails@gmail.com')
    inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[2]/td[2]/input")
    inputElement.send_keys('QprNf3ncs')
    inputElement.submit()
    
    # в строке поиска вставляем символ/слово и ищем
    inputElement = driver.find_element_by_xpath('//*[@id="value(input1)"]')
    inputElement.send_keys('a') # вот тут, пишем, что ищем
    inputElement.submit()
    
    # открываем первую ссыллку в строке поиска в новой вкладке и сохраняем url
    t = '//*[@id="RECORD_1"]/div[3]/div[1]/div/a/value'
    link = driver.find_element_by_xpath(t).click()
    driver.find_element_by_xpath('//*[@id="hidden_section_label"]').click() # кликаем на "show more data"
    url = str(driver.current_url)[:-1]
    
    startPage = 1000
    p = url + str(startPage)
    driver.get(p)
    
    return print("Succeed")

In [5]:
Start(1000, None)

Succeed


In [ ]:
L = [] #создаем итоговый список
# проходим циклом по всем статьям из поиска через url (см. далее)
# чтобы добавить другие статьи из поиска, то можно изменить range на другой (статьи добавятся, старые не сотрутся)
for page in range(1000, 2000):
    try:
        d = {} # словарик для каждой статьи

        # создаем лист для авторов, куда добавлем всех из описания статьи 
        # (видимых на странице <= 10, со скрытыми не хотелось мучаться, так что берем только их)
        # todo: авторы это линки, по ним можно пройти собрать их
        authors = []
        for author in range(10):
            try:
                authors.append(driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[2]/p/a['+str(author)+']').text)
            except:
                pass
        d['authors'] = authors # лист с авторами сохраняем как value для ключа authors словаря d

        # ищем заголовок статьи и также добавляем в словарь
        try:
            title = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[1]/value').text
            d['title'] = title
        except:
            d['title'] = 'NaN'

        # ищем дату выпуска статьи (дата статьи в плохом формате, 
        # так что мб стоит потом пробежаться и поменять или отформатировать тут)
        try:
            year = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[3]/p[2]/value').text
            d['year'] = year
        except:
            d['year'] = 'NaN'

        # ищем заголовк журнала и также добавляем в словарь
        try:
            source = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[3]/p[1]/span/value').text
            d['source'] = source
        except:
            d['source'] = 'NaN'

        # ищем начальную и конечную страницу в журнле у статьи (например: 342-345) и вычитаем из второй первую
        # разницу записываем в словарь
        try:
            pagesNumber = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[3]/div/p[3]/value').text
            pages = str(pagesNumber).split("-")
            try:
                pagesNumber = int(pages[1]) - int(pages[0])
            except:
                pass
            d['pagesNumber'] = pagesNumber
        except:
            d['pagesNumber'] = 'NaN'

        # ищем кол-во цитирований статьи и добавляем в словарь
        try:
            citationsNumber = driver.find_element_by_xpath('//*[@id="sidebar-column1"]/div/div[3]/div/span').text
            d['citationsNumber'] = citationsNumber
        except:
            d['citationsNumber'] = 'NaN'

        # ищем DOI статьи и добавляем в словарь
        try:
            DOI = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[3]/p[2]/value').text
            d['DOI'] = DOI
        except:
            d['DOI'] = 'NaN'

        # ищем url статьи и добавляем в словарь
        d['link'] = driver.current_url

        # ищем абстракт статьи и добавляем в словарь
        try:
            shortDescription = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[4]/p').text
            d['shortDescription'] = shortDescription
        except:
            d['shortDescription'] = 'NaN'

        # создаем лист для ключевых слов, куда добавлем их из описания статьи 
        # (я ограницелся 10тью, так как не видел, чтобы было их больше)
        try:
            keywords = []
            for keyword in range(10):
                try:
                    keywords.append(driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[5]/p[1]/a['+str(keyword)+']').text)
                except:
                    pass
            d['keywords'] = keywords # лист с ключевыми словами сохраняем как value для ключа keywords словаря d
        except:
            d['keywords'] = 'NaN'

        # ищем язык статьи из "show more data" и добавляем в словарь
        try:
            documentLang = driver.find_element_by_xpath('//*[@id="hidden_section"]/div[1]/p[1]').text
            d['documentLang'] = str(documentLang)[9:]
        except:
            d['documentLang'] = 'NaN'

        # ищем паблищера и добавляем в словарь
        try:
            publisher = driver.find_element_by_xpath('//*[@id="records_form"]/div/div/div/div[1]/div/div[3]/p[1]/span/value').text
            d['publisher'] = publisher
        except:
            d['publisher'] = 'NaN'

        # ищем конференцию (если есть) и в словарик conference добавлем title и date
        try:
            conference = {}
            conference_title = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[4]/p[1]/value').text
            conference_date = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[4]/p[3]/value').text
            conference['title'] = conference_title
            conference['date'] = conference_date
            d['conference'] = conference #сам словарик conference передаем как value к ключу conference словаря d
        except:
            d['conference'] = 'NaN'

        # создаем лист для ссылок на другие сатьи
        # (я ограницелся 30тью, так как для большего кол-ва нужно обходить доп. страницу и делать доп. проверку)
        workLinks = []
        for workLink in range(30):
            try:
                workLinks.append(driver.find_element_by_xpath('/html/body/div[1]/div[28]/div[2]/div['+str(workLink)+']/a/span/value').text)
            except:
                pass
        d['workLinks'] = workLinks

        L.append(d) # добавляем словарь d как элемент в итоговый лист L
        print(page, 'is OK')
        driver.find_element_by_xpath('//*[@id="paginationForm"]/span/a[2]').click()
        
    except Exception as e:
        a = 'bad'
        while a!='good':
            try:
                proxy = random.choice(proxy_list)
                print("type error: " + str(e))
                driver.quit()
                Start(page, proxy)
                a ='good'
            except:
                pass

In [ ]:
FINAL.extend(L)

In [ ]:
with open('articles.json', 'w') as f:
    json.dump(FINAL, f)

In [ ]:
driver.close()

In [ ]:
FINAL

# Scraping second try

In [54]:
driver = webdriver.Chrome(executable_path='/home/skvid/Desktop/сети/project_2/chromedriver')
driver.get("http://proxylibrary.hse.ru:2048/login?url=http://isiknowledge.com/wos")

In [55]:
def start(driver):
    # Открываем браузер, вводим логин и пароль
    inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[1]/td[2]/input")
    inputElement.send_keys('marishamails@gmail.com')
    inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[2]/td[2]/input")
    inputElement.send_keys('QprNf3ncs')
    inputElement.submit()
    
    # в строке поиска вставляем символ/слово и ищем
    inputElement = driver.find_element_by_xpath('//*[@id="value(input1)"]')
    inputElement.send_keys('c') # вот тут, пишем, что ищем
    inputElement.submit()
    
    return print("Succeed")

In [56]:
start(driver)

Succeed


In [57]:
def fileLoader(driver, startPage):
    
    endPage = startPage + 499
    
    driver.find_element_by_xpath('//*[@id="exportTypeName"]').click()
    try:
        driver.find_element_by_xpath('//*[@id="saveToMenu"]/li[3]/a').click()
    except:
        pass
    driver.find_element_by_xpath('//*[@id="numberOfRecordsRange"]').click()
    
    input1 = driver.find_element_by_xpath('//*[@id="markFrom"]').clear()
    input2 = driver.find_element_by_xpath('//*[@id="markTo"]').clear()
    
    input1 = driver.find_element_by_xpath('//*[@id="markFrom"]')
    input2 = driver.find_element_by_xpath('//*[@id="markTo"]')
    
    input1.send_keys(str(startPage))
    input2.send_keys(str(endPage))
      
    driver.find_element_by_xpath('//*[@id="select2-saveOptions-container"]').click()
    driver.find_element_by_xpath("//li[contains(text(),'HTML')]").click()
    
    driver.find_element_by_xpath('//*[@id="select2-bib_fields-container"]').click()
    driver.find_element_by_xpath("//li[contains(text(),'Full Record and Cited References')]").click()
    
    driver.find_element_by_xpath('//*[@id="exportButton"]').click()
    
    return print(startPage, endPage, 'done')

In [58]:
fileLoader(driver, 1)

1 500 done


In [59]:
for i in range(180, 200):
    driver.refresh()
    fileLoader(driver, i*500+1)
    sleep(10)

90001 90500 done
90501 91000 done
91001 91500 done
91501 92000 done
92001 92500 done
92501 93000 done
93001 93500 done
93501 94000 done
94001 94500 done
94501 95000 done
95001 95500 done
95501 96000 done
96001 96500 done
96501 97000 done
97001 97500 done
97501 98000 done
98001 98500 done
98501 99000 done
99001 99500 done
99501 100000 done


# Journals scraping try

In [4]:
driver = webdriver.Chrome(executable_path='/home/skvid/Desktop/networks/project_2/chromedriver')
driver.get("https://proxylibrary.hse.ru:3812/JCRJournalHomeAction.action")

In [5]:
# Открываем браузер, вводим логин и пароль
inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[1]/td[2]/input")
inputElement.send_keys('anastasiaozerova36@gmail.com ')
inputElement = driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/div[2]/form/table/tbody/tr[2]/td[2]/input")
inputElement.send_keys('E2VWnyMqf')
inputElement.submit()

In [6]:
data = pd.read_csv('journals.txt', header = None, sep='\t', names=['title', 'id', 'category', 'impactFactor'])
print(data.shape)
data.head()

(13539, 4)


,title,id,category,impactFactor
0,IRANIAN JOURNAL OF NEONATOLOGY,NaN,NaN,NaN
1,IRANIAN JOURNAL OF CHILD NEUROLOGY,NaN,NaN,NaN
2,CANADIAN JOURNAL FOR THE STUDY OF ADULT EDUCATION,NaN,NaN,NaN
3,JOURNAL OF JAPANESE STUDIES,NaN,NaN,NaN
4,JOURNAL OF PERINATAL EDUCATION,NaN,NaN,NaN


In [7]:
def journalLoader(driver, journal_name,):
    
    driver.switch_to.window(driver.window_handles[0])
    driver.refresh()
    search = driver.find_element_by_xpath('//*[@id="search-inputEl"]').clear()
    search = driver.find_element_by_xpath('//*[@id="search-inputEl"]')
    search.send_keys(str(journal_name))
    sleep(5)
    search.send_keys(Keys.ENTER)
    sleep(5)
    driver.switch_to.window(driver.window_handles[1])
    
    try:
        ISSN = driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[2]/div[1]/div[1]').text
        categories = driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[2]/div[2]/div[2]/div[1]/div/div[2]').text

        driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[4]/div[2]/div[1]/div[3]/div[1]/div/div[1]/div[4]').click()
        sleep(5)

        index = {}
        for i in range(1, 100):
            try:
                driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[4]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[4]/div[2]/div[1]').click()
            except:
                pass
            try:
                jcr_0 = driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[4]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div['+str(i)+']/div[2]/div[1]').get_attribute("ymark")
                jcr_1 = driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[4]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div['+str(i)+']/div[2]/div[2]').get_attribute("ymark")
                year = driver.find_element_by_xpath('/html/body/div[2]/jif-webapp/main/home/dashboard/div[5]/div[4]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div['+str(i)+']/div[1]').text
                if list(jcr_0)[-1] == '%':
                    index[year] = jcr_1
                else:
                    index[year] = jcr_0
            except:
                break
    
    except:
        pass
    
    driver.close()
        
    return ISSN, categories, index

In [8]:
for j in range(1, 100):
    try:
        arr = journalLoader(driver, data.loc[j, 'title'])
        data.loc[j, 'id'] = arr[0]
        data.loc[j, 'category'] = arr[1]  
        data.loc[j, 'impactFactor'] = [arr[2]]
    except:
        pass

In [12]:
data.head(5)

,title,id,category,impactFactor
0,IRANIAN JOURNAL OF NEONATOLOGY,NaN,NaN,NaN
1,IRANIAN JOURNAL OF CHILD NEUROLOGY,NaN,NaN,NaN
2,CANADIAN JOURNAL FOR THE STUDY OF ADULT EDUCATION,NaN,NaN,NaN
3,JOURNAL OF JAPANESE STUDIES,NaN,NaN,NaN
4,JOURNAL OF PERINATAL EDUCATION,NaN,NaN,NaN
